# USO DE LA LIBRERÍA LIBMF CON PYTHON

## OBJETIVO 

El objetivo de este notebook es mostrar la aplicación de algunas funciones/ métodos de la librería libmf con python, por lo que primero se importarán los siguientes paquetes. En caso de que no se tenga alguno instalado usar el comando `pip install`.

In [1]:
import sys
import numpy as np
import types
import scipy
from scipy.sparse import random
from scipy import stats
import pandas as pd
import copy
import random
from functools import partial
from colorama import init

import libmf
from libmf import mf

Using file found in /opt/anaconda3/lib/python3.7/site-packages:
/opt/anaconda3/lib/python3.7/site-packages/libmf.cpython-37m-darwin.so


A continuación se crean algunas funciones auxiliares que contienen su respectiva documentación.

### Funciones auxiliares para generar matrices ralas (con muchas entradas nulas o ceros)

In [2]:
def perform(fun, *args):
    """
    Evalua una función con los argumentos indicados
    
    params: fun         Una función
            *args       Parámetros que utiliza la función "fun"
                        (se pueden meter cuantos parámetros tenga 
                        dicha funcion)
    return: fun(*args)  Valor de fun con parámetros *args
    """
    return fun(*args)

In [3]:
def nan_or_value(x, f, *args):
    """
    Devuelve el valor de una función evaluada con sus parámetros
    *args acorde a la variable indicadora x
    
    params: x         boolean, si es 1 tómese valor de la función
                      si no será nan
            f         Una función
            *args     Parámetros que utiliza la función "f"
                      (se pueden meter cuantos parámetros tenga 
                      dicha funcion)
            
    return: 
        perform(f,*args) Si el valor de x es 1 y en otro caso regresa nan
    """
    if(x==1):
        return perform(f,*args)
    else:
        return np.nan

In [4]:
#Con esta vectorización se podrá aplicar a todos los elementos de un np.array la función nan_or_value
nan_or_value_vec=np.vectorize(nan_or_value, otypes=[object])

La siguiente función es la que crea matrices aletorias ralas. Es importante señalar, que por la forma de construcción pudiera no ser eficiente para matrices de dimensiones grandes.

In [5]:
def crear_matriz_rala(n_ren,m_col,prop_elem_difer_nan,func_rand,*args_func_rand):
    """
    Construye matriz rala de n_renxm_col, seleccionando al azar los elementos
    que serán nan cumpliendo que la proporción de éstos es de 
    1-prop_elem_difer_nan, donde los numeros diferentes de nan serán aleatorios
    generados a partir de una función aleatoria que se envíe como parámetro
    (y sus argumentos)
    
    params: n_ren               int número de renglones que tiene la matriz
            m_col               int número de columnas que tiene la matriz
            prop_elem_difer_nan float (0 < prop_elem_difer_nan < 1) indica
                                la proporción de elementos distintos de nan
            func_rand           Función que genera un sólo número aleatorio de
                                tipo float (no arreglos)
            *args_fun_rand      Argumentos necesarios para ejecutar la
                                función fun_rand
            
    return:  M                  np.ndarray que contiene matriz rala
    """
    tot=int(n_ren*m_col)
    k=int(tot*prop_elem_difer_nan)
    #se juntan 2 arreglos, uno de ceros y otro de unos, para después
    #reacomodarlos al azar
    M=np.concatenate((np.zeros(tot-k).astype(int), np.ones(k).astype(int)))
    M=np.array(random.sample(list(M),tot))
    M=M.reshape(n_ren,m_col)
    #Los unos se intercambias por numeros aleatorios y los ceros
    #por nan (o se dejan como cero)
    M=nan_or_value_vec(M,func_rand,*args_func_rand).astype(float)
    return M

### Función que compacta las matrices ralas (quita las entradas que sean na o ceros)

In [6]:
def compactar_matriz_rala(M):
    """
    Pasa una matriz rala a su expresión compacta
    
    params: M  np.ndarray que representa matriz rala
    return: MC np.ndarray matriz en expresión compacta
               (no se reportarán los elementos con valor
               nan)
    """
    if type(M) is not np.ndarray:
        sys.exit('M debe ser de tipo numpy.ndarray')
    elif np.isnan(M).sum()==len(M):
        sys.exit('Todos los elementos de M son nan')
    MC=np.zeros((M[np.isnan(M)==False].size,3))
    k = 0
    n,m=M.shape
    for i in range(n): 
        for j in range(m): 
            if (~np.isnan(M[i,j])): 
                MC[k,0] = i
                MC[k,1] = j
                MC[k,2] = M[i,j] 
                k += 1
    return MC

### Función que crea una matriz aleatoria en el formato que lo requiere la librería LIBMF.

Este formato debe tener en la primer columna el índice de la fila (se podría pensar como el id del usuario), en la segunda el índice de la columna (se podría pensar como el id de la película) y finalmente el valor/calificación.

In [7]:
def crear_matriz_rala_compacta(n_ren, m_col, prop_elem_difer_nan):
    """
    Construye matriz de n_renxm_col, seleccionando al azar los elementos
    que serán distintos de nan. Éstos elementos distintos de nan serán
    números aleatorios entre 1 y 5 (uniforme discreta). Se dice compacta porque
    en esta matriz solo señala las coordenadas o entradas de la matriz que 
    son diferentes a nan
    
    params: n_ren               int número de renglones que tiene la matriz
            m_col               int número de columnas que tiene la matriz
            prop_elem_difer_nan float (0 < prop_elem_difer_nan < 1) indica
                                la proporción de elementos distintos de nan
    return: M  np.ndarray que contiene la matriz 
               (1era columna representa el índice de renglones,
               2da columna representa el índice de columnas,
               3era columna representa el valor del elemento
               o calificación en el contexto de netflix)
    """
    tot=int(n_ren*m_col)
    k=int(tot*prop_elem_difer_nan)
    M=random.sample(range(tot), k)
    columnas=np.mod(M,m_col)
    renglones=np.divide(M,m_col).astype(int)
    aleatorios=np.random.randint(1, 6, k)
    
    M=np.vstack((renglones,columnas,aleatorios)).transpose()


    return M

### Función para generar las muestras de entrenamiento y validación en el formato que lo requiere la librería LIBMF

In [8]:
def muestra_val_ent(df,prop):
    """
    Separa al azar una base dada en 2 bases acorde una proporción de elementos
    dada
    
    params: df    pd.DataFrame (con 3 columnas=['usuario_id','peli_id',
                  'calif_x'])
            prop  float (0 < prop < 1) proporción de id_usuarios y
                  proporción de id_peliculas que se seleccionarán
    
    return: data_test      pd.DataFrame que tiene la base que se usará
                           para prueba (por construcción el número de
                           elementos de esta base no necesariamente coincide
                           con prop*total de elementos de df)
            data_training  pd.DataFrame que tiene la base que se usará
                           para entrenamiento
    """
    #Base de todos los usuarios
    usuarios=pd.DataFrame(df['usuario_id'].unique(), columns=['usuario_id'])
    #Muestreo de usuarios
    valida_usuarios=usuarios.sample(frac=prop)  

    #Base de todas las películas
    peliculas=pd.DataFrame(df['peli_id'].unique(), columns=['peli_id'])
    #Muestreo de películas
    valida_pelis=peliculas.sample(frac=prop)
    
    #df_valida tiene las columnas usuario_id, peli_id y calif, pero
    #sólo incluye los usuarios muestreados en valida_usuarios
    df_valida=pd.merge(df,valida_usuarios, on ='usuario_id', how='inner')
    #data_valida filtra de la base df_valida los pelis_id muestreas
    #en valida_pelis
    data_test=pd.merge(df_valida,valida_pelis, on ='peli_id', how='inner')
    left=pd.merge(df, data_test, how='left',on=['usuario_id', 'peli_id'])
    data_training=left[left.isnull().any(axis=1)][['usuario_id', 'peli_id','calif_x']]
    return data_test, data_training

### Función que imprime el valor de los párametros que se usaron para realizar el entrenamiento.

In [9]:
def imprime_atributos_MF(Obj):
    """
    Imprime los parámetros utilizados para factorización de la matriz contenida
    en Obj, mismos que son atributos del atributo _options
    
    params: Obj  libmf.mf.MF objeto de la librería libmf-python que contiene
                 la información requerida para hacer la factorización deseada
    
    return: None No regresa nada, sólo imprime los parámetros utilizados para
                 hacer la factorización
    """
    if type(Obj) is not libmf.mf.MF:
        sys.exit('A y b deben ser de tipo libmf.mf.MF')
    print('Atributos del modelo:')
    print('fun={0:d}'.format(Obj._options.fun))
    print('k={0:d}'.format(Obj._options.k))
    print('nr_threads={0:d}'.format(Obj._options.nr_threads))
    print('nr_bins={0:d}'.format(Obj._options.nr_bins))
    print('nr_iters={0:d}'.format(Obj._options.nr_iters))
    print('lambda_p1={0:0.4f}'.format(Obj._options.lambda_p1))
    print('lambda_p2={0:0.4f}'.format(Obj._options.lambda_p2))
    print('lambda_q1={0:0.4f}'.format(Obj._options.lambda_q1))
    print('lambda_q2={0:0.4f}'.format(Obj._options.lambda_q2))
    print('eta={0:0.4f}'.format(Obj._options.eta))
    print('do_nmf=',Obj._options.do_nmf)
    print('quiet=',Obj._options.quiet)
    print('copy_data=',Obj._options.copy_data)
    return None

### Función que imprime los datos del modelo entrenado

In [10]:
def imprime_atributos_model(Obj):
    """
    Imprime los atributos del objeto model (definido dentro dentro de la
    librería libmf)
    
    params: Obj  libmf.mf.MFModel objeto de la librería libmf-python que
                 contiene los resultados obtenidos de la corrida del
                 modelo de factorización
                 
    return: None No regresa nada, sólo imprime los resultados contenidos
            en el objeto model, generado con los métodos del objeto MF
    """
    if type(Obj) is not libmf.mf.MFModel:
        sys.exit('A y b deben ser de tipo libmf.mf.MF')
    print('Resultados model:')
    print('fun={0:d}'.format(Obj.fun))
    print('m={0:d}'.format(Obj.m))
    print('n={0:d}'.format(Obj.n))
    print('k={0:d}'.format(Obj.m))
    print('b={0:0.4f}'.format(Obj.b))
    print('P={0:0.4f}'.format(Obj.P.contents.value))
    print('Q={0:0.4f}'.format(Obj.Q.contents.value))
    return None

Con las funciones definidas anteriormente, se procede a hacer uso de ellas para probar las funciones/métodos de la librería LIBMF de python.

En primer lugar, se empezará por crear una matriz rala. Estas matrices son de nuestro interés ya que en el caso de las películas de Netflix, no todos los usuarios ven todas las películas o no todos califican las películas vistas, por lo que si se considera que el índice de las filas es el id de los usuarios y el índice de las columnas es el id de la película, entonces estas matrices contienen muchos "huecos". Por esta razón nos interesa generar ese tipo de matrices.

No obstante lo anterior, debido al numero de usuarios y películas, existen muchos datos, y para no guardar todos los valores de cero (o NA), se opta por expresar y guardar dicha matriz en una forma compacta, la cual contiene por cada calificación el usuario que la generó y la película a la que corrresponde (es decir, es una matriz con 3 columnas).

A continuación, se crea una matriz rala de 20 filas, 5 columnas, porcentaje de entradas diferentes de na del 25% (25 entradas diferentes de na) y con generación de enteros aleatorios entre el 1 y 5. 

No se omite señalar, que la generación de números enteros puede cambiarse por alguna otra función que se requiera, por ejemplo una constante o bernoulli.


In [11]:
M=crear_matriz_rala(20,5,0.25,stats.randint.rvs,*[1,6])

print('Matriz M:\n',M)
print('Tamaño M:\n',M.shape)
print('Numero de elementos diferentes de nan:\n',M[np.isnan(M)==False].size)

Matriz M:
 [[nan nan  2. nan nan]
 [nan nan nan  4. nan]
 [nan nan nan  1. nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [ 3. nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [ 5.  3.  3. nan nan]
 [nan nan nan nan nan]
 [nan  2. nan  3. nan]
 [nan nan nan nan nan]
 [nan nan  3. nan nan]
 [ 3.  4.  2. nan nan]
 [ 3. nan  5. nan nan]
 [nan  1. nan  5. nan]
 [nan nan  2. nan  5.]
 [nan  3.  3. nan nan]
 [nan  3.  2.  3.  1.]]
Tamaño M:
 (20, 5)
Numero de elementos diferentes de nan:
 25


A continuación se compacta la matriz anterior, es decir, solo considerando las entradas donde se tenga un valor y 
donde la primer columna representa el índice de la fila, la segunda el índice de la columna y la tercera el valor.

**El formato descrito anteriormente es el que solicita la librería libmf.**

En este ejemplo se tendrá una matriz de 25 filas (las diferentes de na)

In [12]:
MC=compactar_matriz_rala(M)
print('Expresión compacta de M (MC):\n',MC)
print('Tamaño MC:\n',MC.shape)

Expresión compacta de M (MC):
 [[ 0.  2.  2.]
 [ 1.  3.  4.]
 [ 2.  3.  1.]
 [ 6.  0.  3.]
 [ 9.  0.  5.]
 [ 9.  1.  3.]
 [ 9.  2.  3.]
 [11.  1.  2.]
 [11.  3.  3.]
 [13.  2.  3.]
 [14.  0.  3.]
 [14.  1.  4.]
 [14.  2.  2.]
 [15.  0.  3.]
 [15.  2.  5.]
 [16.  1.  1.]
 [16.  3.  5.]
 [17.  2.  2.]
 [17.  4.  5.]
 [18.  1.  3.]
 [18.  2.  3.]
 [19.  1.  3.]
 [19.  2.  2.]
 [19.  3.  3.]
 [19.  4.  1.]]
Tamaño MC:
 (25, 3)


Dado que la librería requiere el formato descrito anteriormente, usaremos directamente la función "crea_matriz_rala_compacta" indicandole el número de filas, de columnas y el porcentaje de entradas distinto de na para que directamente devuelva una matriz compacta en el diseño que lo requiere la librería.

Dentro de las funciones de libmf existe una que se llama "generate_test_data" que genera las matrices en este formato, sin embargo, notamos que al usar ésta podría darse el caso de que una persona califique a una misma película con dos o más calificaciones diferentes, tal como se muestra en los registros marcados en rojo del siguiente ejemplo:

In [13]:
#El 1er parámetro indica que se seleccionará un valor
#de entre 0 y 19 para la 1er columna (que representa el índice
#de los renglones de la matriz rala que expresa); el 2do
#parámetro indica que se seleccionara al azar valores entre
#0 y 4 (que representa el índice de las columnas de la matriz
#rala que expresa); el 3er parámetro indica el número de elementos
#a generar 

np.random.seed(102)
MC=mf.generate_test_data(20, 5, 25)
print('Matriz generada con mf.generate_test_data(20,5,25):\n')
for i in range(MC.shape[0]):
    if MC[i,0]==0:
        print('\033[1;30;41m',MC[i,:])
    else:
        print('\033[0;0;0m',MC[i,:])
print('Tamaño MC:\n',MC.shape)

Matriz generada con mf.generate_test_data(20,5,25):

 [0.         1.         0.16637107]
 [19.          4.          0.30636304]
 [14.          3.          0.14721805]
 [18.          4.          0.22291025]
 [18.          1.          0.53951716]
 [15.          3.          0.74892651]
 [2.         2.         0.42831194]
 [11.          3.          0.10467911]
 [9.         3.         0.75337018]
 [8.         5.         0.59815912]
 [13.          0.          0.48677848]
 [8.         4.         0.97530286]
 [9.         2.         0.80230277]
 [15.          3.          0.33301479]
 [7.         5.         0.54855018]
 [4.         5.         0.31180599]
 [16.          4.          0.06230988]
 [6.         1.         0.90313856]
 [18.          0.          0.86054759]
 [3.        3.        0.6296915]
 [0.         1.         0.21655643]
 [0.        1.        0.5557169]
 [16.          5.          0.10932898]
 [6.         5.         0.55083566]
 [7.        0.        0.5812561]
Tamaño MC:
 (25, 3)


In [14]:
MC=crear_matriz_rala_compacta(20,5,0.25)
print('Expresión compacta de M (MC) ordenada por renglon:\n',MC[np.argsort(MC[:,0])])
print('Tamaño MC:\n',MC.shape)

Expresión compacta de M (MC) ordenada por renglon:
 [[ 0  0  1]
 [ 0  1  2]
 [ 1  1  5]
 [ 2  3  1]
 [ 4  4  5]
 [ 4  3  2]
 [ 4  1  3]
 [ 5  4  5]
 [ 5  2  3]
 [ 7  1  4]
 [ 8  4  2]
 [10  1  3]
 [11  3  5]
 [11  1  1]
 [13  1  4]
 [14  0  4]
 [15  4  4]
 [15  2  4]
 [16  4  2]
 [16  2  4]
 [17  3  5]
 [17  1  1]
 [18  3  1]
 [19  4  3]
 [19  0  3]]
Tamaño MC:
 (25, 3)


## CLASE MF DE LIBMF-PYTHON


A continuación se describe la clase base de la librería libmf en python que es la de **MF**. Con esta clase se realiza la factorización de la matriz R en otras dos matrices P y Q (entrenamiento del modelo) y se realizan las predicciones que se necesitan para tener un sistema de recomendación.

Se describe un resumen de su documentación:

* Clase: MF<br/>
Atributos:<br/>  
            .model
            ._options 
            .i
            .j
 Métodos:<br/>
          .fit -> < Objetivo: factorize the i x j data matrix X into (j, k) (k, i) 
                  <           sized matrices stored in MF.model
                  < Parámetro X: (n, 3) shaped numpy array [known index and values 
                  <              of the data matrix]

          .predict -> < Objetivo: assuming we have already run the fit method, predict
                      <           the values at certain indices of the data matrix
                      < Parámetro X: (n, 2) shaped numpy array
                      < Regresa: numpy array of length n

          .mf_cross_validation -> < Objetivo: Realizar cross-validation
                                  < Parámetro X: (n, 3)
                                  < Parámetro folds: number of train / test splits
                                  < Regresa: score across all folds 

          .q_factors -> < Objetivo: Obtener la matriz Q

          .p_factors -> < Objetivo:  Obtener la matriz P
          


En lo siguiente se muestran casos de su uso:

In [15]:
#objeto de tipo MF para obtener la factorización deseada
mf_netflix=mf.MF()

#veamos algunas de las propiedades del objeto mf_netflix
print('Tipo de objeto: ',type(mf_netflix))
print('Instance variables: ',mf_netflix.__dict__)

Tipo de objeto:  <class 'libmf.mf.MF'>
Instance variables:  {'model': None, '_options': <libmf.mf.MFParam object at 0x1a19ae2320>, 'i': None, 'j': None}


En la definición del objeto anterior no se especificó ningún parámetro, por lo que se usarán los valores de default de cada uno de ellos para entrenar el modelo (factorización de la matriz R).

Veamos cuáles son estos valores default.

In [16]:
imprime_atributos_MF(mf_netflix)

Atributos del modelo:
fun=0
k=8
nr_threads=12
nr_bins=26
nr_iters=20
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.1000
do_nmf= False
quiet= False
copy_data= True


Observemos también que esos parámetros se encuentran definidos dentro de los atributos del objeto:

In [17]:
print('Atributos: ', dir(mf_netflix._options))

Atributos:  ['__class__', '__ctypes_from_outparam__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_b_base_', '_b_needsfree_', '_fields_', '_objects', 'copy_data', 'do_nmf', 'eta', 'fun', 'k', 'lambda_p1', 'lambda_p2', 'lambda_q1', 'lambda_q2', 'nr_bins', 'nr_iters', 'nr_threads', 'quiet']


Sin embargo, en la definición del objeto se puede especificar el valor de cada parámetro para poder obtener la mejor factorización posible (sin llegar al sobreajuste)

In [18]:
mf_netflix=mf.MF(fun=0,k=10,nr_threads=12,nr_bins=26,nr_iters=20,lambda_p1=0.04,lambda_p2=0.0,lambda_q1=0.04,lambda_q2=0.0,eta=0.1,do_nmf=False,quiet=False,copy_data=True)
imprime_atributos_MF(mf_netflix)


Atributos del modelo:
fun=0
k=10
nr_threads=12
nr_bins=26
nr_iters=20
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.1000
do_nmf= False
quiet= False
copy_data= True


In [19]:
mf_netflix=mf.MF(eta=0.11,nr_iters=21, k=5)

imprime_atributos_MF(mf_netflix)

Atributos del modelo:
fun=0
k=5
nr_threads=12
nr_bins=26
nr_iters=21
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.1100
do_nmf= False
quiet= False
copy_data= True


### MÉTODO FIT DE LA CLASE MF

A este método se le tiene que pasar la matriz de entrenamiento (con el formato especificado anteriormente) y su objetivo es realizar la factorización de la matriz R, es decir, obtener P y Q.

Primero generemos la matriz de entrenamiento y validación.

In [20]:
ME=crear_matriz_rala_compacta(20,5,0.40)
MV=crear_matriz_rala_compacta(20,5,0.15)

Posteriormente, a estas dos muestras les daremos el siguiente tratamiento.

Es importante resaltar que solo vamos a considerar las primeras dos columnas (sin los valores/calificaciones) de "MV" dado que así lo requiere el método predict de esta clase para justo obtener las predicciones.

In [21]:
MV_index=MV[:,0:2]
MC_entrena=np.empty((40,3,))
MC_entrena[:]=np.nan
MC_valida=np.empty((15,2,))
MC_valida[:]=np.nan
for i in range(40):
    for j in range(3):
        MC_entrena[i,j]=float(ME[i,j])
        if i<15 and j<2:
            MC_valida[i,j]=float(MV_index[i,j])

In [22]:
MC_entrena

array([[11.,  3.,  5.],
       [11.,  4.,  4.],
       [10.,  4.,  5.],
       [13.,  0.,  5.],
       [18.,  4.,  3.],
       [ 2.,  4.,  1.],
       [ 0.,  1.,  5.],
       [15.,  4.,  4.],
       [17.,  4.,  2.],
       [10.,  3.,  2.],
       [ 7.,  3.,  2.],
       [ 1.,  3.,  2.],
       [ 9.,  0.,  1.],
       [15.,  0.,  1.],
       [16.,  3.,  3.],
       [12.,  4.,  5.],
       [11.,  2.,  1.],
       [11.,  0.,  5.],
       [ 4.,  1.,  3.],
       [ 7.,  4.,  5.],
       [ 1.,  2.,  5.],
       [19.,  2.,  2.],
       [12.,  2.,  4.],
       [18.,  2.,  5.],
       [12.,  0.,  3.],
       [ 3.,  1.,  1.],
       [17.,  0.,  3.],
       [16.,  4.,  4.],
       [14.,  4.,  1.],
       [ 6.,  1.,  2.],
       [ 8.,  3.,  1.],
       [14.,  2.,  4.],
       [13.,  4.,  4.],
       [15.,  1.,  4.],
       [ 5.,  0.,  1.],
       [ 4.,  0.,  5.],
       [ 6.,  2.,  3.],
       [ 6.,  4.,  3.],
       [10.,  1.,  4.],
       [19.,  4.,  3.]])

Aplicamos el método fit  a la matriz de entrenamiento para obtener la factorización, en este ejemplo con los valores que se le asignan a los parámetros por default.

In [23]:
mf_netflix = mf.MF()
mf_netflix.fit(MC_entrena)

### MÉTODO VALIDACIÓN CRUZADA DE LA CLASE MF

A este método se le tiene que pasar la matriz de entrenamiento (con el formato especificado anteriormente) y el número de folds a realizar, en caso de que no se requiera el de default (5) y devuelve el score.


In [24]:
print('Validación cruzada con 5 folds: ',mf_netflix.mf_cross_validation(MC_entrena))

Validación cruzada con 5 folds:  2.0833972022911675


### MÉTODO PREDICT DE LA CLASE MF

A este método se le tiene que pasar la matriz de validación (con el formato especificado anteriormente) y devuelve un vector con las predicciones, con número de entradas igual al número de filas de la matriz de validación.


In [25]:
pred=mf_netflix.predict(MC_valida)
pred

array([3.8494096, 3.15     , 3.6547995, 3.15     , 0.870219 , 3.15     ,
       3.2371597, 2.7428324, 2.5692482, 1.1390778, 0.9577841, 3.15     ,
       3.6973605, 3.15     , 1.8846557], dtype=float32)

In [26]:
print('Los valores verdaderos de MC_valida son:',MV[:,2])

Los valores verdaderos de MC_valida son: [2 3 4 1 1 4 1 4 2 2 2 3 3 2 1]


### MÉTODO Q_FACTORS DE LA CLASE MF

Con este método se obtiene la información relacionada a la matriz Q

In [27]:
Q=mf_netflix.q_factors()
print('Tamaño Q:',Q.shape)
print('Matriz Q:\n',Q)

Tamaño Q: (5, 8)
Matriz Q:
 [[ 0.92554075  0.13283074  0.42496398  1.330313    1.0461099   0.6722095
   1.3373908   0.0312187 ]
 [ 0.73527586  0.96832305  0.2293356   0.62338495  0.7322146   1.3061383
   0.4250327   1.0249578 ]
 [ 1.135339    0.22270554  1.1328886   0.43923956  1.6109912   0.7988438
  -0.1837712   0.7920804 ]
 [ 0.36625618  0.58380556  0.2211914   1.0482459   0.5146589   0.2496472
   1.2645755   0.06796654]
 [ 0.77889955  1.481756    0.28612852  0.72553825  0.9580456   1.0354813
   0.8840778   0.61594087]]


### MÉTODO P_FACTORS DE LA CLASE MF

Con este método se obtiene la información relacionada a la matriz P

In [28]:
P=mf_netflix.p_factors()
print('Tamaño P:',P.shape)
print('Matriz P:\n',P)

Tamaño P: (20, 8)
Matriz P:
 [[ 6.9187248e-01  7.4295610e-01  3.2220769e-01  7.1507949e-01
   6.8657750e-01  1.1249733e+00  4.2700648e-01  8.9246488e-01]
 [ 8.6928946e-01  2.5296515e-01  7.7806997e-01  6.1749625e-01
   1.0608798e+00  6.9095242e-01  8.2889527e-02  5.2232605e-01]
 [ 1.0159652e-01  9.5493324e-02  3.1206089e-01  3.4236616e-01
   1.4865229e-01  1.2671061e-02  2.0785305e-01  7.3285259e-02]
 [ 8.1352569e-02  1.7207696e-01  1.1362135e-01  2.8049186e-01
   2.3960888e-01  1.8999358e-01  1.9498384e-01  9.5334323e-03]
 [ 5.7262123e-01  1.0561044e-01  5.0034291e-01  1.0361123e+00
   9.5461488e-01  5.5116171e-01  8.3611852e-01  1.7007367e-01]
 [ 2.0440505e-01  5.9684724e-02  2.7887881e-01  1.3237080e-01
   2.3662038e-01  0.0000000e+00  1.5884811e-01  2.6819012e-01]
 [ 5.5866617e-01  4.5294949e-01  3.2025740e-01  1.6269134e-01
   8.6215168e-01  3.1905976e-01  2.1563727e-01  1.1667450e-01]
 [ 6.5535396e-01  7.6827675e-01  4.8706904e-01  5.9386545e-01
   7.1326995e-01  6.3563645e-01  5

En el siguiente ejemplo se indica que se realice la factorización no negativa, dado que se observa que en la matriz P anterior hay entradas negativas, que el número de iteraciones sea igual a 30 y que el número de factores latentes sea igual a 5.

In [29]:
mf_netflix = mf.MF(do_nmf=True, nr_iters=30, k=5)
mf_netflix.fit(MC_entrena)

In [30]:
pred=mf_netflix.predict(MC_valida)
pred

array([4.3507032 , 3.15      , 3.40761   , 3.15      , 0.83365715,
       3.15      , 3.1568196 , 3.309884  , 3.2439501 , 1.0403211 ,
       0.9771335 , 3.15      , 3.990199  , 3.15      , 1.1636386 ],
      dtype=float32)

In [31]:
print('Los valores verdaderos de MC_valida son:',MV[:,2])

Los valores verdaderos de MC_valida son: [2 3 4 1 1 4 1 4 2 2 2 3 3 2 1]


In [32]:
Q=mf_netflix.q_factors()
print('Tamaño Q:',Q.shape)
print('Matriz Q:\n',Q)

Tamaño Q: (5, 5)
Matriz Q:
 [[5.7679170e-01 6.9930995e-01 1.5103151e-01 1.7653044e+00 1.7104127e+00]
 [6.4528549e-01 9.0039623e-01 1.7551240e+00 7.1617931e-01 5.4637313e-01]
 [4.9602729e-01 1.1153817e+00 1.4103149e+00 1.5069052e+00 1.8975930e-01]
 [7.6336819e-01 9.3359560e-01 7.6578872e-04 1.0700930e+00 9.1725242e-01]
 [1.6091663e+00 1.7668656e-01 1.6382926e+00 8.1129640e-01 9.2833775e-01]]


In [33]:
P=mf_netflix.p_factors()
print('Tamaño P:',P.shape)
print('Matriz P:\n',P)

Tamaño P: (20, 5)
Matriz P:
 [[0.49245852 0.95934874 1.3640375  1.1355462  0.8962948 ]
 [0.74261147 0.67487544 1.0667952  1.1416166  0.39055252]
 [0.03595683 0.27906382 0.27610534 0.14800441 0.3048714 ]
 [0.40171784 0.05316391 0.18500192 0.13231511 0.4140084 ]
 [0.43175536 0.6130884  0.38441077 1.1495609  1.239467  ]
 [0.19832885 0.10852543 0.1832922  0.30321103 0.12648599]
 [0.56873876 0.38887435 0.35523087 0.9256571  0.53503114]
 [0.96073395 0.37673736 1.282539   0.7612451  0.4748032 ]
 [0.0977657  0.18049772 0.40306738 0.31192225 0.43091834]
 [0.         0.         0.15747258 0.24699706 0.30245635]
 [0.96224    0.20370173 1.3432026  0.73487204 0.5111935 ]
 [0.96691465 0.6736087  0.00632663 0.96218747 1.1770265 ]
 [0.931409   0.44646236 1.2183614  0.8374481  0.4067721 ]
 [0.62927043 0.53896785 0.629067   1.1563523  1.1169927 ]
 [0.         1.0614103  0.37191823 0.9469902  0.11275017]
 [0.7759942  0.57015043 1.5292933  0.04327297 0.00583384]
 [0.8968872  0.83259726 0.7246322  0.807056